In [ ]:
#libraries
import requests as r
import pandas as pd
import numpy as np
from itertools import product
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#credentials, key removed
url='https://api.yelp.com/v3/businesses/search'
key=''
headers={'Authorization':'Bearer %s' %key}

In [ ]:
#parameters, scraping sample size to check out data
parameters= {'location': '919 Milam Street, Houston TX, 77002', 'limit': 3, 'term': 'romantic', 'radius': 20000}
response=r.get(url, headers=headers, params=parameters)
data=response.json()
data

In [ ]:
#1000 limit and 50 steps
#multiple addresses, 500 listings this time
addresses= ['919 Milam Street, Houston TX, 77002']
offset=np.arange(0,1000,50)

#list of tuples, combination of address and offset
tuples=list(product(addresses, offset))

#viewing 5
tuples[:5]

In [ ]:
#for loop, store results into list of list, turn into DF
listings=[]
#future DF columns
cols=['Name', 'Reviews', 'Rating', 'Address', 'Phone', 'Latitude', 'Longitude','Category']

#for loop
for address, step in tuples:
    search_parameters={
        'location': address,
        'term': 'romantic', 
        'radius': 20000,
        'limit': 50, 
        'offset': step}
    resp=r.get(url, headers=headers, params=search_parameters)
    raw_data=resp.json()
    for business in raw_data['businesses']:
        name = business['name']
        reviews = business['review_count']
        rating = business['rating']
        location = business['location']['display_address'][0]
        phone = business['display_phone']
        latitude = business['coordinates']['latitude']
        longitude = business['coordinates']['longitude']
        category = business['categories'][0]['title']
        listings.append([name, reviews, rating, location, phone, latitude, longitude, category])
    df = pd.DataFrame.from_records(listings, index='Name', columns=cols)

#checking out the intial dataset
print(f'Total Records : {len(df)}')
df.head()

In [ ]:
#EDA
display(df.info())
display(df.describe())

In [ ]:
#Check out places with most reviews and highest ratings
df=df.sort_values(by='Reviews', ascending=False)
display(df.head())
df=df.sort_values(by='Rating', ascending=False)
display(df.head())

In [ ]:
#visualization
g=sns.catplot(x='Category', data=df, kind='count')
plt.xticks(rotation=90)
g.fig.set_size_inches(15,15)

In [ ]:
pd.set_option("display.max_rows", None, "display.max_columns", None)
print(df.groupby(['Category']).agg(['mean', 'min', 'max', 'std', 'count']).Rating)